# 신경망 학습 최적화

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'New Gulim'
rcParams['font.size'] = 10
rcParams['axes.unicode_minus'] = False

# 1 매개변수 갱신 - Optimizer

### 1.1 확률적 경사 하강법 - SGD

<img src="./images/e_6.1.png" width="180"/>

In [ ]:
# SGD 클래스 구현
class SGD:
    """확률적 경사 하강법（Stochastic Gradient Descent）"""
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key] 

### 1.2 SGD의 단점

- 함수 정의
<img src="./images/e_6.2.png" width="200"/>

- 그래프와 등고선
<img src="./images/fig_6-1.png" width="500"/>

- 함수의 기울기
<img src="./images/fig_6-2.png" width="400"/>

- SGD에 의한 최적화 갱신 경로: 초기값 (-7.0, 2.0)
<img src="./images/fig_6-3.png" width="400"/>

### 1.3 모멘텀 - Momentum

<img src="./images/e_6.3.png" width="180"/>
<img src="./images/e_6.4.png" width="150"/>

- 모멘텀의 이미지
<img src="./images/fig_6-4.png" width="500"/>

In [ ]:
# 모멘텀 클래스 구현
class Momentum:
    """모멘텀 SGD"""
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():                                
                self.v[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key] 
            params[key] += self.v[key]

- 모멘텀에 의한 최적화 갱신 경로: 초기값 (-7.0, 2.0)
<img src="./images/fig_6-5.png" width="400"/>

### 1.4 AdaGrad

<img src="./images/e_6.5.png" width="180"/>
<img src="./images/e_6.6.png" width="180"/>

In [ ]:
# AdaGrad 클래스 구현
class AdaGrad:
    """AdaGrad"""
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
            
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

- AdaGrad에 의한 최적화 갱신 경로: 초기값 (-7.0, 2.0)
<img src="./images/fig_6-6.png" width="400"/>

### 1.5 Adam

In [ ]:
class Adam:
    """Adam (http://arxiv.org/abs/1412.6980v8)"""
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        
        self.iter += 1
        lr_t  = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)         
        
        for key in params.keys():
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])
            
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)
            

- Adam에 의한 최적화 갱신 경로: 초기값 (-7.0, 2.0)
<img src="./images/fig_6-7.png" width="400"/>

### 1.6 Optimizer 비교

#### 1.6.1 다변수 함수

- 함수
<img src="./images/e_6.2.png" width="200"/>

<img src="./images/fig_6-8.png" width="500"/>

- 코드 구현

In [ ]:
from collections import OrderedDict
from common.optimizer import *

# 함수 정의
def f(x, y):
    return x**2 / 20.0 + y**2

# 도함수 정의
def df(x, y):
    return x / 10.0, 2.0*y

# 초기값
init_pos = (-7.0, 2.0)
params = {}
params['x'], params['y'] = init_pos[0], init_pos[1]
grads = {}
grads['x'], grads['y'] = 0, 0

optimizers = OrderedDict()
optimizers["SGD"] = SGD(lr=0.95)
optimizers["Momentum"] = Momentum(lr=0.1)
optimizers["AdaGrad"] = AdaGrad(lr=1.5)
optimizers["Adam"] = Adam(lr=0.3)

idx = 1

for key in optimizers:
    optimizer = optimizers[key]
    x_history = []
    y_history = []
    params['x'], params['y'] = init_pos[0], init_pos[1]
    
    for i in range(30):
        x_history.append(params['x'])
        y_history.append(params['y'])
        
        grads['x'], grads['y'] = df(params['x'], params['y'])
        optimizer.update(params, grads)
    

    x = np.arange(-10, 10, 0.01)
    y = np.arange(-5, 5, 0.01)
    
    X, Y = np.meshgrid(x, y) 
    Z = f(X, Y)
    
    # 외곽선 단순화
    mask = Z > 7
    Z[mask] = 0
    
    # 그래프 그리기
    plt.subplot(2, 2, idx)
    idx += 1
    plt.plot(x_history, y_history, 'o-', color="red")
    plt.contour(X, Y, Z)
    plt.ylim(-10, 10)
    plt.xlim(-10, 10)
    plt.plot(0, 0, '+')
    #colorbar()
    #spring()
    plt.title(key)
    plt.xlabel("x")
    plt.ylabel("y")
    
plt.show()

#### 1.6.2  MNIST 데이터셋

- MNIST 데이터셋 학습 진행 과정
<img src="./images/fig_6-9.png" width="500"/>

- MNIST 학습 진행 과정 코드 구현

In [ ]:
%%time
from dataset.mnist import load_mnist
from common.util import smooth_curve
from common.multi_layer_net import MultiLayerNet
from common.optimizer import *

# 0. MNIST 데이터 읽기==========
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

train_size = x_train.shape[0]
batch_size = 128
max_iterations = 2000

# 1. 실험용 설정==========
optimizers = {}
optimizers['SGD'] = SGD()
optimizers['Momentum'] = Momentum()
optimizers['AdaGrad'] = AdaGrad()
optimizers['Adam'] = Adam()
#optimizers['RMSprop'] = RMSprop()

networks = {}
train_loss = {}
for key in optimizers.keys():
    networks[key] = MultiLayerNet(
        input_size=784, hidden_size_list=[100, 100, 100, 100],
        output_size=10)
    train_loss[key] = []    

# 2. 훈련 시작==========
for i in range(max_iterations):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    for key in optimizers.keys():
        grads = networks[key].gradient(x_batch, t_batch)
        optimizers[key].update(networks[key].params, grads)
    
        loss = networks[key].loss(x_batch, t_batch)
        train_loss[key].append(loss)
    
    if i % 100 == 0:
        print( "===========" + "iteration:" + str(i) + "===========")
        for key in optimizers.keys():
            loss = networks[key].loss(x_batch, t_batch)
            print(key + ":" + str(loss))

In [ ]:
# 3. 그래프 그리기==========
markers = {"SGD": "o", "Momentum": "x", "AdaGrad": "s", "Adam": "D"}
x = np.arange(max_iterations)
for key in optimizers.keys():
    plt.plot(x, smooth_curve(train_loss[key]), marker=markers[key], markevery=100, label=key)
plt.xlabel("iterations")
plt.ylabel("loss")
plt.ylim(0, 1)
plt.legend()
plt.show()

# 2 가중치 초기값
- sigmoid, tanh: Xavier 초기값
- ReLU:  He 초기값

### 2.1 은닉층의 활성화값 분포


In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
def ReLU(x):
    return np.maximum(0, x)

In [ ]:
def tanh(x):
    return np.tanh(x)

In [ ]:
def plot_activation( w_init_case = 1, act_func = sigmoid):
    input_data = np.random.randn(1000, 100)  # 1000개의 데이터
    node_num = 100                           # 각 은닉층의 노드(뉴런) 수
    hidden_layer_size = 5                    # 은닉층이 5개
    activations = {}                         # 이곳에 활성화 결과를 저장

    x = input_data

    for i in range(hidden_layer_size):
        if i != 0:
            x = activations[i-1]

        # 초깃값을 다양하게 바꿔가며 실험해보자！
        if w_init_case   == 1:
            w = np.random.randn(node_num, node_num) * 1
        elif w_init_case == 2:
            w = np.random.randn(node_num, node_num) * 0.01
        elif w_init_case == 3:
            w = np.random.randn(node_num, node_num) * np.sqrt(1.0 / node_num)
        elif w_init_case == 4:
            w = np.random.randn(node_num, node_num) * np.sqrt(2.0 / node_num)
        else:
            w = np.random.randn(node_num, node_num) * 1

        a = np.dot(x, w)

        # 활성화 함수도 바꿔가며 실험해보자！
        z = act_func(a)
        # z = sigmoid(a)
        # z = ReLU(a)
        # z = tanh(a)

        activations[i] = z

    # 히스토그램 그리기
    for i, a in activations.items():
        plt.subplot(1, len(activations), i+1)
        plt.title(str(i+1) + '-layer')
        if i != 0: plt.yticks([], [])
        plt.hist(a.flatten(), 30, range=(0,1))
        
    plt.show()

#### 2.1.1 가중치를 표준편차가 1인 정규분포로 초기화

In [ ]:
# w: 표준편차 1인 정규분포로 초기화, sigmoid 사용
plot_activation(w_init_case=1, act_func=sigmoid)

#### 2.1.2 가중치를 표준편차가 0.01인 정규분포로 초기화

In [ ]:
# w: 표준편차 0.01인 정규분포로 초기화, sigmoid 사용
plot_activation(w_init_case=2, act_func=sigmoid)

#### 2.1.3 Xavier 초기값
- 사비에르 글로로트(Xavier Glorot), 요슈아 벤지오(Yoshua Bengio)

<img src="./images/fig_6-12.png" width="400"/>

In [ ]:
# w: Xavier 초기화, sigmoid 사용
plot_activation(w_init_case=3, act_func=sigmoid)

#### 2.1.4 He 초기값
- 카이밍 히(Kaîming He)
- ReLU 사용시 최적화

- w: 표준편차가 $\sqrt{\frac{2}{n}}$ 인 정규 분포로 초기화

#### ReLU 사용시 가중치 초기화

In [ ]:
# w: 표준편차 0.01인 정규분포로 초기화, ReLU 사용
plot_activation(w_init_case=2, act_func=ReLU)

In [ ]:
# w: Xavier 초기화, ReLU 사용
plot_activation(w_init_case=3, act_func=ReLU)

In [ ]:
# w: He 초기화, ReLU 사용
plot_activation(w_init_case=4, act_func=ReLU)

### 2.2 가중치 초기값 비교: MNIST 데이터셋

- MNIST 데이터셋 학습 진행 과정: 기중치 초기값에 따른 비교
<img src="./images/fig_6-15.png" width="500"/>

- MNIST 학습 진행 과정 코드 구현: 기중치 초기값에 따른 비교

In [ ]:
%%time
from dataset.mnist import load_mnist
from common.util import smooth_curve
from common.multi_layer_net import MultiLayerNet
from common.optimizer import SGD

# 0. MNIST 데이터 읽기==========
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

train_size = x_train.shape[0]
batch_size = 128
max_iterations = 2000

# 1. 실험용 설정==========
weight_init_types = {'std=0.01': 0.01, 'Xavier': 'sigmoid', 'He': 'relu'}
optimizer = SGD(lr=0.01)

networks = {}
train_loss = {}
for key, weight_type in weight_init_types.items():
    networks[key] = MultiLayerNet(input_size=784, hidden_size_list=[100, 100, 100, 100],
                                  output_size=10, weight_init_std=weight_type)
    train_loss[key] = []

# 2. 훈련 시작==========
for i in range(max_iterations):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    for key in weight_init_types.keys():
        grads = networks[key].gradient(x_batch, t_batch)
        optimizer.update(networks[key].params, grads)
    
        loss = networks[key].loss(x_batch, t_batch)
        train_loss[key].append(loss)
    
    if i % 100 == 0:
        print("===========" + "iteration:" + str(i) + "===========")
        for key in weight_init_types.keys():
            loss = networks[key].loss(x_batch, t_batch)
            print(key + ":" + str(loss))

In [ ]:
# 3. 그래프 그리기==========
markers = {'std=0.01': 'o', 'Xavier': 's', 'He': 'D'}
x = np.arange(max_iterations)
for key in weight_init_types.keys():
    plt.plot(x, smooth_curve(train_loss[key]), marker=markers[key], markevery=100, label=key)
plt.xlabel("iterations")
plt.ylabel("loss")
plt.ylim(0, 2.5)
plt.legend()
plt.show()

# 3 배치 정규화 - Batch Normalization

### 3.1 배치 정규화 알고리즘

- 학습 속도 개선
- 초기값 의존도 낮춤
- 오버피팅 억제

- 배치 정규화를 사용한 신경망의 예
<img src="./images/fig_6-16.png" width="600"/>

- 배치 정규화 수식
<img src="./images/e_6.7.png" width="200"/>

- 배치 정규화의 확대(scale), 이동(shift)
- $\gamma = 1, \beta = 0$ 으로 학습 시작, 학습 진행에 따라서 조정
<img src="./images/e_6.8.png" width="150"/>

### 3.2 배치 정규화 효과

- 배치 정규화 효과: 학습 속도 향상
<img src="./images/fig_6-18.png" width="500"/>

- 가중치 초기값에 따른 학습 진행
- 실선: 배치 정규화 사용, 점선: 배치 정규화 상용하지 않음
<img src="./images/fig_6-19.png" width="500"/>

# 4 오버피팅 - Over fitting

### 4.1 오버피팅된 학습 진행 과정

In [ ]:
%%time
from dataset.mnist import load_mnist
from common.multi_layer_net import MultiLayerNet
from common.optimizer import SGD

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

# 오버피팅을 재현하기 위해 학습 데이터 수를 줄임
x_train = x_train[:300]
t_train = t_train[:300]

# weight decay（가중치 감쇠） 설정 =========================
weight_decay_lambda = 0 # weight decay를 사용하지 않을 경우
#weight_decay_lambda = 0.1
# ==========================================================

network = MultiLayerNet(input_size=784, hidden_size_list=[100, 100, 100, 100, 100, 100], output_size=10,
                        weight_decay_lambda=weight_decay_lambda)
optimizer = SGD(lr=0.01) # 학습률이 0.01인 SGD로 매개변수 갱신

max_epochs = 201
train_size = x_train.shape[0]
batch_size = 100

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)
epoch_cnt = 0

for i in range(1000000000):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grads = network.gradient(x_batch, t_batch)
    optimizer.update(network.params, grads)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print("epoch:" + str(epoch_cnt) + ", train acc:" + str(train_acc) + ", test acc:" + str(test_acc))

        epoch_cnt += 1
        if epoch_cnt >= max_epochs:
            break

In [ ]:
# 그래프 그리기==========
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, train_acc_list, marker='o', label='train', markevery=10)
plt.plot(x, test_acc_list, marker='s', label='test', markevery=10)
plt.title('오버피팅된 학습 과정')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

### 4.2 가중치 감소 - L2 규제

In [ ]:
%%time
from dataset.mnist import load_mnist
from common.multi_layer_net import MultiLayerNet
from common.optimizer import SGD

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

# 오버피팅을 재현하기 위해 학습 데이터 수를 줄임
x_train = x_train[:300]
t_train = t_train[:300]

# weight decay（가중치 감쇠） 설정 =========================
#weight_decay_lambda = 0 # weight decay를 사용하지 않을 경우
weight_decay_lambda = 0.1
# ==========================================================

network = MultiLayerNet(input_size=784, hidden_size_list=[100, 100, 100, 100, 100, 100], output_size=10,
                        weight_decay_lambda=weight_decay_lambda)
optimizer = SGD(lr=0.01) # 학습률이 0.01인 SGD로 매개변수 갱신

max_epochs = 201
train_size = x_train.shape[0]
batch_size = 100

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)
epoch_cnt = 0

for i in range(1000000000):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grads = network.gradient(x_batch, t_batch)
    optimizer.update(network.params, grads)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print("epoch:" + str(epoch_cnt) + ", train acc:" + str(train_acc) + ", test acc:" + str(test_acc))

        epoch_cnt += 1
        if epoch_cnt >= max_epochs:
            break

In [ ]:
# 그래프 그리기==========
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, train_acc_list, marker='o', label='train', markevery=10)
plt.plot(x, test_acc_list, marker='s', label='test', markevery=10)
plt.title('L2 규제 적용')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

### 4.3 드롭아웃 - Dropout

- 뉴런을 무작위로 삭제하여 신호 전달을 차단
<img src="./images/fig_6-22.png" width="500"/>

#### 드롭아웃을 적용한 신경망 학습 과정

In [ ]:
%%time
from dataset.mnist import load_mnist
from common.multi_layer_net_extend import MultiLayerNetExtend
from common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

# 오버피팅을 재현하기 위해 학습 데이터 수를 줄임
x_train = x_train[:300]
t_train = t_train[:300]

# 드롭아웃 사용 유무와 비울 설정 ========================
use_dropout = True  # 드롭아웃을 쓰지 않을 때는 False
dropout_ratio = 0.2
dropout_ratio = 0.15
# =======================================================

network = MultiLayerNetExtend(input_size=784, hidden_size_list=[100, 100, 100, 100, 100, 100],
                              output_size=10, use_dropout=use_dropout, dropout_ration=dropout_ratio)
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=301, mini_batch_size=100,
                  optimizer='sgd', optimizer_param={'lr': 0.01}, verbose=True)
trainer.train()

train_acc_list, test_acc_list = trainer.train_acc_list, trainer.test_acc_list

In [ ]:
# 그래프 그리기==========
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, marker='o', label='train', markevery=10)
plt.plot(x, test_acc_list, marker='s', label='test', markevery=10)
plt.title('드롭아웃을 적용한 신경망 학습 과정: dropout_ratio = 0.15')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

# 5 하이퍼파라미터 튜닝

- lr: 학습률
- weight decay: L2 규제
<img src="./images/fig_6-24.png" width="500"/>

- Best-1 (val acc:0.83) | lr:0.0092, weight decay: 3.86e-07
- Best-2 (val acc:0.78) | lr:0.0095, weight decay: 6.04e-07

# 정리

- 매개변수 갱신 방법에는 확률적 경사 하강법(SGD) 외에도 모멘텀, AdaGrad, Adam 등이 있다.
- 가중치 초깃값을 정하는 방법은 올바른 학습을 히는 데 매우 중요하다.
- 가중치의 초깃값으로는 'Xavier 초깃값'과 'He 초깃값'이 효괴적이다.
- 배치 정규화를 이용하면 학습을 빠르게 진행할 수 있으며, 초기값에 영향을 덜 받게 된다.
- 오버피팅을 억제하는 정규화 기술로는 기중치 감소와 드롭아웃이 있다.
- 하이퍼따라미터 값 탐색은 최적 값이 존재할 법한 범위를 점차좁히면서 하는 것이 효과적이다.

---

In [ ]:
# End of file